In [37]:
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from ipywidgets import interact
import pickle
from IPython.display import clear_output
plt.rcParams['figure.dpi'] = 200

# Order of files
"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",

"/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"

In [38]:
# file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

file_headers = [#"/Volumes/Flash/combined_data/far_right_up_and_back",
                #"/Volumes/Flash/combined_data/far_left_up_and_back",
                #"/Volumes/Flash/combined_data/up_and_back3",
                #"/Volumes/Flash/combined_data/sweep1",
                #"/Volumes/Flash/combined_data/sweep2",
                #"/Volumes/Flash/combined_data/random",
                "/Volumes/Flash/combined_data/module1_fullext1",
                "/Volumes/Flash/combined_data/module1_fullext2",
                "/Volumes/Flash/combined_data/module1_fullext3",
                #"/Volumes/Flash/combined_data/module1_fullext4",
                "/Volumes/Flash/combined_data/module2_fullext1",
                "/Volumes/Flash/combined_data/module2_fullext2",
                "/Volumes/Flash/combined_data/module2_fullext3",
                "/Volumes/Flash/combined_data/module2_fullext4"]
                #"/Volumes/Flash/combined_data/s_curve1",
                #"/Volumes/Flash/combined_data/s_curve1_part2",
                #"/Volumes/Flash/combined_data/s_curve2",
                #"/Volumes/Flash/combined_data/s_curve2_part2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [39]:
#poly_df_list = []
marker_df_list = []
data_df_list = []
idx_df_list = []
idx_file_df_list = []
data_idx_df_list = []

running_idx = 0
end_idx = []

for file_i, header in enumerate(file_headers):
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    #poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    #poly_df = pd.read_csv(poly_file)
    
    idx_array = running_idx+data_df.index.values
    idx_df = pd.DataFrame(idx_array, columns=['global_index'])
    end_idx.append(idx_array[-1])
    running_idx = idx_array[-1] + 1   

    idx_file_array = np.zeros(data_df.shape[0], dtype=int) + file_i
    idx_file_df = pd.DataFrame(idx_file_array, columns=['file_index'])
    
    data_idx_array = data_df.index.values
    data_idx_df = pd.DataFrame(data_idx_array, columns=['data_index'])
    
#     zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
#     print(marker_df)
#     print(zero_indices)
    
#     data_df = data_df.drop(data_df.index[zero_indices])
#     marker_df = marker_df.drop(marker_df.index[zero_indices])
#     #poly_df = poly_df.drop(poly_df.index[zero_indices])
#     idx_df = idx_df.drop(idx_df.index[zero_indices])
#     idx_file_df = idx_file_df.drop(idx_file_df.index[zero_indices])
#     data_idx_df = data_idx_df.drop(data_idx_df.index[zero_indices])
    #print(marker_df)
    data_df_list.append(data_df)
    marker_df_list.append(marker_df)
    #poly_df_list.append(poly_df)
    idx_df_list.append(idx_df)
    idx_file_df_list.append(idx_file_df)
    data_idx_df_list.append(data_idx_df)

total_data_df = pd.concat(data_df_list).reset_index(drop=True)
total_marker_df = pd.concat(marker_df_list).reset_index(drop=True)
#total_poly_df = pd.concat(poly_df_list).reset_index(drop=True)
total_idx_df = pd.concat(idx_df_list).reset_index(drop=True)
total_idx_file_df = pd.concat(idx_file_df_list).reset_index(drop=True)
total_data_idx_df = pd.concat(data_idx_df_list).reset_index(drop=True)

global_cont_idx_array = np.arange(total_data_df.shape[0], dtype=int)
total_global_cont_idx_df = pd.DataFrame(global_cont_idx_array, columns=['global_cont1_index'])

diff1_arr = total_data_df["M1-PL"] - total_data_df["M1-PR"] #left minus right
diff2_arr = total_data_df["M2-PL"] - total_data_df["M2-PR"]
total_data_df["M1-Diff"] = diff1_arr
total_data_df["M2-Diff"] = diff2_arr

m1_al_in_act = total_data_df["M1-AL-IN"] * total_data_df["PUMP"] * total_data_df["GATE"]
m1_ar_in_act = total_data_df["M1-AR-IN"] * total_data_df["PUMP"] * total_data_df["GATE"]
m2_al_in_act = total_data_df["M2-AL-IN"] * total_data_df["PUMP"] * total_data_df["GATE"]
m2_ar_in_act = total_data_df["M2-AR-IN"] * total_data_df["PUMP"] * total_data_df["GATE"]

total_data_df["M1-AL-IN-ACT"] = m1_al_in_act
total_data_df["M1-AR-IN-ACT"] = m1_ar_in_act
total_data_df["M2-AL-IN-ACT"] = m2_al_in_act
total_data_df["M2-AR-IN-ACT"] = m2_ar_in_act

training_data_list = [total_data_df, total_marker_df, total_idx_df, total_idx_file_df, total_data_idx_df, total_global_cont_idx_df]
all_df = pd.concat(training_data_list, axis=1)

In [40]:
def get_groups(df, idx_col):
    return df.groupby(df[idx_col].diff().ne(1).cumsum()).groups

In [41]:
lookback_columns = [
"TIME",
"M1-PL",
"M1-PR",
"M2-PL",
"M2-PR",
"M1-AL-IN",
"M1-AL-OUT",
"M1-AR-IN",
"M1-AR-OUT",
"M2-AL-IN",
"M2-AL-OUT",
"M2-AR-IN",
"M2-AR-OUT",
"PUMP",
"GATE",
"M1-Diff",
"M2-Diff",
"M1-AL-IN-ACT",
"M1-AR-IN-ACT",
"M2-AL-IN-ACT",
"M2-AR-IN-ACT",
"global_index",
"file_index",
"data_index",
"global_cont1_index"]

lookback = 3
stride = 7
threshold = lookback * stride

fresh_arr = np.zeros(all_df.shape[0], dtype=int)
for i in range(lookback):
    for col in lookback_columns:
        fresh_df = pd.DataFrame(fresh_arr, columns=[col + str(i)])
        all_df = pd.concat([all_df, fresh_df], axis=1)

In [42]:
data_groups = get_groups(all_df, "data_index")
for dg in data_groups:
    start_idx = data_groups[dg][0]
    for idx in data_groups[dg]:
        if (idx - start_idx) >= threshold:
            for i in range(lookback):
                new_lookback_columns = []
                for colname in lookback_columns:
                    new_lookback_columns.append(colname + str(i))
                back_idx = idx - (stride*(i+1))
                back_data = all_df.loc[back_idx, lookback_columns]
                all_df.loc[idx, new_lookback_columns] = back_data.values

In [43]:
print(all_df["TIME0"])

0         0.000
1         0.000
2         0.000
3         0.000
4         0.000
         ...   
3516    211.698
3517    212.168
3518    212.644
3519    213.138
3520    213.604
Name: TIME0, Length: 3521, dtype: float64


In [44]:
zero_indices = list(all_df[all_df["TIME0"] == 0].index)
print(zero_indices)
all_df = all_df.drop(all_df.index[zero_indices]).reset_index(drop=True)

global_cont2_idx_array = np.arange(all_df.shape[0], dtype=int)
total_global_cont2_idx_df = pd.DataFrame(global_cont2_idx_array, columns=['global_cont2_index'])
all_df = pd.concat([all_df, total_global_cont2_idx_df], axis=1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084]


In [45]:
all_df.to_csv("data/all_df_basic.csv")

In [20]:
poly_heads = ["3a0","3a1","3a2","3a3","4a0","4a1","4a2","4a3","4a4","5a0","5a1","5a2","5a3","5a4","5a5"]
for p in poly_heads:
    print("head: " + p + ", min: " + str(all_df[p].min()) + ", max: " + str(all_df[p].max()))

KeyError: '3a0'

# Normalization Thresholds

In [46]:
MX_min = -15
MX_max = 15
MY_min = 0
MY_max = 40

P_min = 95
P_max = 121

P_diff_min = -26
P_diff_max = 26

# d_min = 20
# d_max = 40

# a0_min = -0.019
# a0_max = 0.014

# a1_min = -0.3
# a1_max = 0.22

# a2_min = -0.11
# a2_max = 0.17

# a3_min = -0.027
# a3_max = 0.016

# a4_min = -0.0007
# a4_max = 0.0014

# a5_min = -0.000023
# a5_max = 0.0000085

norm_bounds = pd.DataFrame()
for i in range(11):
    norm_bounds["M" + str(i) + "X"] = [MX_min, MX_max]
    norm_bounds["M" + str(i) + "Y"] = [MY_min, MY_max]

# for i in range(3):
#     norm_bounds[str(i+3) + "a0"] = [a0_min, a0_max]
# for i in range(3):
#     norm_bounds[str(i+3) + "a1"] = [a1_min, a1_max]
# for i in range(3):
#     norm_bounds[str(i+3) + "a2"] = [a2_min, a2_max]
# for i in range(3):
#     norm_bounds[str(i+3) + "a3"] = [a3_min, a3_max]
# for i in range(2):
#     norm_bounds[str(i+4) + "a4"] = [a4_min, a4_max]
# norm_bounds["5a5"] = [a5_min, a5_max]
# norm_bounds["d"] = [d_min, d_max]

norm_bounds["M1-PL"] = [P_min, P_max]
norm_bounds["M1-PR"] = [P_min, P_max]
norm_bounds["M2-PL"] = [P_min, P_max]
norm_bounds["M2-PR"] = [P_min, P_max]
norm_bounds["M1-Diff"] = [P_diff_min, P_diff_max]
norm_bounds["M2-Diff"] = [P_diff_min, P_diff_max]
for i in range(lookback):
    norm_bounds["M1-PL" + str(i)] = [P_min, P_max]
    norm_bounds["M1-PR" + str(i)] = [P_min, P_max]
    norm_bounds["M2-PL" + str(i)] = [P_min, P_max]
    norm_bounds["M2-PR" + str(i)] = [P_min, P_max]
    norm_bounds["M1-Diff" + str(i)] = [P_diff_min, P_diff_max]
    norm_bounds["M2-Diff" + str(i)] = [P_diff_min, P_diff_max]

In [47]:
norm_bounds.to_csv("data/norm_bounds_basic.csv")

In [48]:
all_df_norm = all_df.copy()
for c in norm_bounds:
    min_val = norm_bounds.loc[0,c]
    max_val = norm_bounds.loc[1,c]
    all_df_norm[c] = (all_df_norm[c] - min_val) / (max_val - min_val)

In [49]:
all_df_norm.to_csv("data/all_df_basic_norm.csv")